# <strong>Bitcoin Recurrent Neural Network<strong>
### Justin Marlor & Habit Blunk
##### *Colorado State University*

This is our notebook that automatically copies data from [this dataset hosted on Kaggle](https://www.kaggle.com/datasets/mczielinski/bitcoin-historical-data), then throws it into various neural networks and predicts the price of Bitcoin.

To run it:

1. Run the script located in this repository at `./env-script`. This will set up your virtual environment. 
2. Run `source ./venv/bin/activate`. This will put you in the virtual environment we have set up, so this notebook can be run on any machine so long as it has Python 3.x and can install the dependencies at `./dependencies.txt`.
3. Paste this into `~/.config/kaggle/kaggle.json`:
    ```json
    {
      "username": "justinmarlor",
      "key": "b98017f9291bfa83686f6c6780d38e04"
    }
    ```
4. Execute each cell in sequence.

#### Cell 1: Imports and TimeSeries class

In [ ]:
import pandas as pd
import subprocess
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.autograd import Variable
from torch.nn import GRU, RNN
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

class TimeSeriesDataset(torch.utils.data.Dataset):
    def __init__(self, tensor, seq_length, target_idx):
        self.tensor = tensor
        self.seq_length = seq_length
        self.target_idx = target_idx

    def __len__(self):
        return len(self.tensor) - self.seq_length

    def __getitem__(self, idx):
        seq = self.tensor[idx:idx + self.seq_length]
        target = self.tensor[idx + self.seq_length, self.target_idx]
        return seq, target 

#### Cell 2: Grabbing and preprocessing data

In [ ]:
result = subprocess.run(['bash', './add-run-kaggle-bitcoin'], capture_output=True,text=True)

print(result.stdout)
print(result.stderr)

if result.returncode == 0:
    df = pd.read_csv("kaggle-bitcoin/upload/btcusd_1-min_data.csv", dtype={"Volume": float}, low_memory=False)
    df['datetime'] = pd.to_datetime(df['Timestamp'].astype('Int64'), unit='s', errors='coerce')
    display(df)

### Reducing the data to 1 Close value per day

In [ ]:
df = df[df['datetime'].dt.time == pd.to_datetime('23:59').time()]
df

PCA: We use PCA as a strategy to discover noise in our dataset, allowing us to remove unnecessary features in our dataset. This PCA shows us that there is a very similar correlation value between each of the 'open', 'high', 'low', 'close' columns. The trading volume is the other feature that has a strong correlation to the price of bitcoin. We will proceed to train the models using the two features, Close and Volume.

In [ ]:
X = df
X = df.drop(columns=['Timestamp', 'datetime'])

# Standardizing data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

loadings = pd.DataFrame(pca.components_.T, 
                        columns=['PC1', 'PC2'], 
                        index=X.columns)
print(loadings)

In [ ]:
# Removal of Redundant Columns
df = df.drop(columns=['Open','High','Low'])
df

#### Cell 3: plotting current dataset

In [ ]:
# Plot 1: Close price
plt.figure(figsize=(10, 5))
plt.plot(df['datetime'], df['Close'], label='close', color='orange')
plt.xlabel('datetime')
plt.ylabel('price')
plt.title('close price over time')
plt.legend()
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

# Plot 2: Volume
plt.figure(figsize=(10, 5))
plt.plot(df['datetime'], df['Volume'], label='volume', color='green')
plt.xlabel('datetime')
plt.ylabel('volume')
plt.title('volume over time')
plt.legend()
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

#### Cell 4: Create tensor and define functions

In [ ]:
tensor = torch.tensor(X.values, dtype=torch.float32)
close_idx = df.columns.get_loc('Close')

def create_sequences(tensor, seq_length, target_idx):
  sequences = []
  targets = []

  for i in range(len(tensor) - seq_length):
    seq = tensor[i:i + seq_length]
    target_value =  tensor[i + seq_length, target_idx]
    sequences.append(seq)
    targets.append(target_value) 
  return torch.stack(sequences), torch.tensor(targets).unsqueeze(1)

seq_length = 60
batch_size = 64
dataset = TimeSeriesDataset(tensor, seq_length, close_idx)

train_size = int(0.95 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

def train_model(model, train_loader, val_loader, num_epochs=100, lr=1e-3):
  loss_fn = nn.MSELoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
  
  mses = []
   
  if isinstance(num_epochs, list):
    for n in num_epochs:
      for epoch in range(n):
        model.train()
        total_loss = 0
        for batch_X, batch_y in train_loader:
          pred = model(batch_X)
          loss = loss_fn(pred.squeeze(), batch_y)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          total_loss += loss.item()
        model.eval()
        val_loss = 0
        with torch.no_grad():
          for val_X, val_y in val_loader:
            val_pred = model(val_X)
            val_loss += loss_fn(val_pred.squeeze(), val_y).item()
      mses.append(total_loss / len(train_loader)) 
  else:
    for epoch in range(num_epochs):
      model.train()
      total_loss = 0
      for batch_X, batch_y in train_loader:
        pred = model(batch_X)
        loss = loss_fn(pred.squeeze(), batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
      model.eval()
      val_loss = 0
      with torch.no_grad():
        for val_X, val_y in val_loader:
          val_pred = model(val_X)
          val_loss += loss_fn(val_pred.squeeze(), val_y).item()
    mses.append(total_loss / len(train_loader)) 
  return mses 


#### Cell 5: RNN model training class

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim=1):
        super(RNNModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first=True, nonlinearity='relu')
        self.fc = nn.Linear(hidden_dim, 1)
    
    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :]) 
        return out

#### Cell 6: Training RNN Model + storing data into .pth file

In [ ]:
rnn_model = RNNModel(input_dim=tensor.shape[1], hidden_dim=64)
epochs = [2, 4, 8, 16, 32, 64, 128]
mses = train_model(rnn_model, train_loader, val_loader, num_epochs=epochs) 
torch.save(rnn_model.state_dict(), "rnn_model.pth")
print(f"number of epochs that gives the lowest MSE: {epochs[mses.index(min(mses))]}")

#### Cell 7: To load trained RNN data

In [ ]:
rnn_model = RNNModel(input_dim=tensor.shape[1], hidden_dim=64)
rnn_model.load_state_dict(torch.load("rnn_model.pth"))

#### Cell 8: Recursive prediction method

In [ ]:
def predict_future(model, input_seq, steps, target_idx):
    model.eval()
    predictions = []
    seq = input_seq.clone()

    with torch.no_grad():
        for _ in range(steps):
            pred = model(seq.unsqueeze(0))
            pred_value = pred.item()
            predictions.append(pred_value)

            # Create the next sequence by shifting and adding the prediction
            new_step = seq[-1].clone()
            new_step[target_idx] = pred_value  # Only update the target (Close price)
            seq = torch.cat((seq[1:], new_step.unsqueeze(0)), dim=0)

    return predictions

#### Cell 9: How many values we will predict with our trained model.

In [ ]:
last_seq = tensor[-seq_length:]  # Get the last sequence from the dataset
future_steps = 100  # our chosen number of predictions
normalized_predictions = predict_future(rnn_model, last_seq, steps=future_steps, target_idx=close_idx)
print(normalized_predictions)
print(set(normalized_predictions))

#### Cell 10: LSTM model training class

In [ ]:
class LSTMModel(nn.Module):
  def __init__(self, input_dim, hidden_dim, layer_dim=1):
    super(LSTMModel, self).__init__()
    self.hidden_dim = hidden_dim
    self.layer_dim = layer_dim
    self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True) 
    self.fc = nn.Linear(hidden_dim, 1)

  def forward(self, x): 
    out, _ = self.lstm(x)
    out = self.fc(out[:, -1, :])
    return out

#### Cell 11: Training LSTM against dataset, then saving it in a `*.pth` file

In [ ]:
lstm_model = LSTMModel(input_dim = tensor.shape[1], hidden_dim=64)
mses = train_model(lstm_model, train_loader, val_loader, num_epochs=epochs)
torch.save(lstm_model.state_dict(), "lstm_model.pth")
print(f"number of epochs that gives the lowest MSE: {epochs[mses.index(min(mses))]}")

#### Loading trained LSTM Model data

In [ ]:
lstm_model = LSTMModel(input_dim=tensor.shape[1], hidden_dim=64)
lstm_model.load_state_dict(torch.load("lstm_model.pth"))

In [ ]:
last_seq = tensor[-seq_length:]  # Get the last sequence from the dataset
future_steps = 100  # our chosen number of predictions
normalized_predictions = predict_future(lstm_model, last_seq, steps=future_steps, target_idx=close_idx)
print(normalized_predictions)
print(set(normalized_predictions))